In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'/Users/kooloosh/Desktop/practice/kaggle/Bank-Churn-Binary-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    sourse_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from ChurnClassification.constants import *
from ChurnClassification.utils.common import read_yaml, create_directories


In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            sourse_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config



In [7]:
import os 
import urllib.request as request
import zipfile
from ChurnClassification import logger
from ChurnClassification.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.sourse_url,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} is downloading! info: {headers}')

        else:
            logger.info(f'{filename} already exists! with size: {get_size(self.config.local_data_file)}')

    
    def unzip_file(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        logger.info(f'file is unzipped at {self.config.unzip_dir}')

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()

except Exception as e:
    logger.exception(e)
    raise e

[2024-01-15 17:45:11,584: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-15 17:45:11,587: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-15 17:45:11,587: INFO: common: created directory at: artifacts]
[2024-01-15 17:45:11,589: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-15 17:45:13,698: INFO: 1418177886: artifacts/data_ingestion/data.zip is downloading! info: Connection: close
Content-Length: 7001621
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d6244bb2aa5fc1e47c29cdf84f5c2a1f8f92f08fa3c562ff90a65a275e16fefc"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0CF4:E10AD:76BA08:7C5D80:65A544F8
Accept-Ranges: bytes
Date: Mon, 15 Jan 2024 14:45:13 GMT
Via: 1.1 varnish
X-Served-By: cache-sof1510029-SOF
X-Cache: MISS
X-Cac